In [1]:

import builtins
import glob
import os
import zipfile
from PIL import Image
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchinfo import summary
import numpy as np
#import pytorchcv


default_device = 'cuda' if torch.cuda.is_available() else 'cpu'

#from pytorchcv import load_mnist, train, plot_results, plot_convolution, display_dataset
#load_mnist(batch_size=64) 

KeyboardInterrupt: 

우선 MNIST 데이터셋을 불러옵니다. 이 함수 

In [ ]:
def load_fashion_mnist(batch_size=64):
    # Fashion MNIST 학습 데이터셋 불러오기
    fashion_train = torchvision.datasets.FashionMNIST('./data', download=True, train=True, transform=transforms.ToTensor())
    
    # Fashion MNIST 테스트 데이터셋 불러오기
    fashion_test = torchvision.datasets.FashionMNIST('./data', download=True, train=False, transform=transforms.ToTensor())
    
    # 학습 데이터셋 로더 정의
    train_loader = torch.utils.data.DataLoader(fashion_train, batch_size=batch_size)
    
    # 테스트 데이터셋 로더 정의
    test_loader = torch.utils.data.DataLoader(fashion_test, batch_size=batch_size)
    
    return train_loader, test_loader

: 

아래 코드는 위 함수의 테스트 코드

In [ ]:
train_loader, test_loader = load_fashion_mnist()

# 데이터셋 확인을 위해 예시로 첫 번째 배치의 데이터를 출력
dataiter = iter(train_loader)
images, labels = next(dataiter, 0)
print(images.shape)  # 이미지 배치의 크기 출력
print(labels.shape)

: 

In [ ]:
import matplotlib.pyplot as plt

# load_mnist 함수 호출하여 데이터 로드
train_loader, test_loader = load_fashion_mnist(batch_size=64)

# 첫 번째 배치의 이미지와 레이블 가져오기
images, labels = next(iter(train_loader))

# 이미지 출력하기
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(images[i].squeeze(), cmap='gray')
    plt.title(f'Label: {labels[i]}')
    plt.axis('off')
plt.show()


: 

In [ ]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = nn.NLLLoss()): 
    # 이 함수는 여러 매개변수를 받는데, net은 학습할 신경망 모델, dataloader는 데이터 로더, lr은 학습률(기본값 0.01), 
    # optimizer는 최적화 도구(기본값은 None), loss_fn은 손실 함수로 기본적으로 Negative Log Likelihood Loss를 사용
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr) # 최적화 도구가 제공되지 않았다면, Adam 최적화 도구를 사용하여 신경망의 매개변수를 최적화하며, 학습률은 lr로 설정
    net.train() # 모델을 학습 모드로 설정합니다. 이는 일부 신경망 계층(예: 드롭아웃 계층)이 학습과 평가 모드에서 다르게 동작하기 때문에 필요
    total_loss,acc,count = 0,0,0 # 총 손실, 정확도, 처리한 샘플 수를 초기화
    for features,labels in dataloader: # 데이터 로더로부터 특징(feature)과 레이블(label)을 반복적으로 가져옴
        optimizer.zero_grad() # 최적화 도구의 모든 기울기를 0으로 초기화하는데 새로운 가중치 업데이트를 위해 필수
        lbls = labels.to(default_device) # 레이블을 기본 계산 장치(예: GPU)로 이동
        out = net(features.to(default_device)) # 특징을 같은 장치로 이동시킨 후, 신경망을 통해 예측을 수행
        loss = loss_fn(out,lbls) #cross_entropy(out,labels) 예측 결과와 레이블을 이용해 손실을 계산
        loss.backward() # 손실에 대한 기울기를 계산
        optimizer.step() # 계산된 기울기를 이용해 신경망의 가중치를 업데이트
        total_loss+=loss # 총 손실을 누적
        _,predicted = torch.max(out,1) # 예측된 결과 중 가장 높은 확률을 가진 클래스를 선택
        acc+=(predicted==lbls).sum() # 정확하게 예측된 수를 누적
        count+=len(labels) # 처리된 레이블의 수를 누적
    return total_loss.item()/count, acc.item()/count # 평균 손실과 정확도를 반환

: 

In [ ]:
# 파이토치(PyTorch)를 사용하여 간단한 신경망 모델을 정의하고, 그 구조를 요약해서 출력하는 과정을 설명

import torch.nn as nn # 파이토치의 신경망 모듈(nn)을 임포트
from torchsummary import summary # 설치된 torchsummary에서 summary 함수를 임포트

class OneConv(nn.Module): # nn.Module을 상속받는 OneConv라는 새로운 클래스를 정의하는데 nn.Module은 파이토치에서 모든 신경망 모듈의 기본 클래스
    def __init__(self): # 클래스의 생성자로, OneConv 객체가 생성될 때 초기화 과정을 정의
        super(OneConv, self).__init__() # 상위 클래스인 nn.Module의 생성자를 호출하여 클래스를 적절히 초기화
        self.conv = nn.Conv2d(in_channels=1,out_channels=9,kernel_size=(5,5)) # 입력 채널이 1개, 출력 채널이 9개인 2D 컨볼루션 레이어를 정의하고, 이를 객체의 conv 속성으로 할당
        self.flatten = nn.Flatten() # 다차원 입력을 1차원 배열로 변환하는 Flatten 레이어를 정의하고, flatten 속성으로 할당
        self.fc = nn.Linear(5184,10) # 5,184개의 입력 특성을 10개의 출력 특성으로 변환하는 완전 연결 레이어(또는 선형 레이어)를 정의하고, fc 속성으로 할당

    def forward(self, x): # 모델의 순전파를 정의하는 메서드로 입력 텐서 x가 모델을 통과하는 과정을 설명
        if x.dim() == 5 and x.size(2) == 1:  # 입력 x의 차원이 예상치 못한 추가 차원을 포함하는 경우, 해당 차원을 제거
            x = x.squeeze(2)
        x = nn.functional.relu(self.conv(x)) # 정의된 컨볼루션 레이어를 입력 x에 적용한 후 ReLU 활성화 함수를 적용
        x = self.flatten(x) # ReLU의 출력을 flatten 레이어를 통해 평탄화
        x = nn.functional.log_softmax(self.fc(x),dim=1) # 평탄화된 벡터를 fc 레이어에 적용하고, 결과를 로그 소프트맥스 함수를 통해 처리하는데 이 함수는 다중 클래스 분류 문제에 대한 확률을 계산할 때 사용

        return x  # The final processed tensor x, which contains the log probabilities of the classes, is returned from the forward method. This output can be used by a loss function during training to compute the error and update the model weights


# 모델 인스턴스화
model = OneConv()

# train_epoch 함수 호출
train_loss, train_acc = train_epoch(model, train_loader)

# 결과 출력
print(f"Training Loss: {train_loss}, Training Accuracy: {train_acc}")


# Print the summary of the model
#summary(model,input_size=(1,1,28,28)) # 생성된 모델 net의 요약 정보를 출력하는데 여기서 input_size는 모델이 기대하는 입력 데이터의 크기를 (채널 수, 높이, 너비) 형식으로 지정

: 

In [ ]:
def validate(net, dataloader,loss_fn=nn.NLLLoss()): # validate 함수를 정의하고, 세 개의 매개변수를 받는데 net은 평가할 신경망 모델, dataloader는 평가 데이터셋을 로딩하는 데이터 로더, loss_fn은 손실 함수로 기본값은 Negative Log Likelihood Loss
    net.eval() # 모델을 평가(evaluation) 모드로 설정하는데 이 모드에서는 모델의 학습 과정에만 적용되는 특정 기능들(예: 드롭아웃)이 비활성화
    count,acc,loss = 0,0,0 # 총 처리한 데이터의 수, 정확히 예측된 데이터의 수, 그리고 총 손실을 0으로 초기화
    with torch.no_grad(): # 기울기 계산이 수행되지 않는데 평가 시에는 모델의 가중치가 갱신되지 않음
        for features,labels in dataloader: # 데이터 로더에서 데이터 배치를 반복적으로 가져오는데 각 배치는 features (특징 데이터)와 labels (레이블 데이터)로 구성
            lbls = labels.to(default_device) # 레이블을 기본 계산 장치(예: GPU)로 이동
            out = net(features.to(default_device)) # 특징 데이터도 같은 계산 장치로 이동한 후, 신경망 모델을 통해 예측을 수행
            loss += loss_fn(out,lbls) # 예측 결과와 실제 레이블을 사용하여 손실을 계산하고, 총 손실에 누적
            pred = torch.max(out,1)[1] # 신경망의 출력에서 가장 높은 값을 가진 클래스의 인덱스를 추출하여 예측 결과로 사용
            acc += (pred==lbls).sum() # 예측이 정확했던 샘플의 수를 누적
            count += len(labels) # 처리한 레이블의 총 수를 누적
    return loss.item()/count, acc.item()/count # 평균 손실과 평균 정확도를 계산하여 반환

: 

In [ ]:
model = OneConv()

# train_epoch 함수 호출
train_loss, train_acc = validate(model, train_loader)

# 결과 출력
print(f"Training Loss: {train_loss}, Training Accuracy: {train_acc}")

: 